# 최종 코드

In [2]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import getcwd
import numpy as np
from PIL import Image

import imgaug as ia
from imgaug import augmenters as iaa

ia.seed(1)

def read_xml_annotation(root, image_id):
    in_file = open(os.path.join(root, image_id), encoding='UTF8')
    tree = ET.parse(in_file)
    root = tree.getroot()
    bndboxlist = []
    for object in root.findall('object'): 
        bndbox = object.find('bndbox')  
        xmin = int(bndbox.find('xmin').text)
        xmax = int(bndbox.find('xmax').text)
        ymin = int(bndbox.find('ymin').text)
        ymax = int(bndbox.find('ymax').text)
        bndboxlist.append([xmin,ymin,xmax,ymax])
    bndbox = root.find('object').find('bndbox')
    return bndboxlist

def change_xml_annotation(root, image_id, new_target):
    new_xmin = new_target[0]
    new_ymin = new_target[1]
    new_xmax = new_target[2]
    new_ymax = new_target[3]
    in_file = open(os.path.join(root, str(image_id) + '.xml'), encoding='UTF8') 
    tree = ET.parse(in_file)
    xmlroot = tree.getroot()
    object = xmlroot.find('object')
    bndbox = object.find('bndbox')
    xmin = bndbox.find('xmin')
    xmin.text = str(new_xmin)
    ymin = bndbox.find('ymin')
    ymin.text = str(new_ymin)
    xmax = bndbox.find('xmax')
    xmax.text = str(new_xmax)
    ymax = bndbox.find('ymax')
    ymax.text = str(new_ymax)
    tree.write(os.path.join(root, str(image_id) + "_aug" + '.xml'))

def change_xml_list_annotation(root, image_id, new_target,saveroot,id):
    in_file = open(os.path.join(root, str(image_id) + '.xml'), encoding='UTF8') 
    tree = ET.parse(in_file)
    xmlroot = tree.getroot()
    index = 0
    for object in xmlroot.findall('object'):  
        bndbox = object.find('bndbox')  

        new_xmin = new_target[index][0]
        new_ymin = new_target[index][1]
        new_xmax = new_target[index][2]
        new_ymax = new_target[index][3]

        xmin = bndbox.find('xmin')
        xmin.text = str(new_xmin)
        ymin = bndbox.find('ymin')
        ymin.text = str(new_ymin)
        xmax = bndbox.find('xmax')
        xmax.text = str(new_xmax)
        ymax = bndbox.find('ymax')
        ymax.text = str(new_ymax)

        index = index + 1
    tree.write(os.path.join(saveroot, str(image_id) + "_aug_" + str(id) + '.xml'))


def mkdir(path):
    path = path.strip()
    path = path.rstrip("\\")
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        os.makedirs(path)
        print(path + '성공')
        return True
    else:
        print(path + '이미 존재')
        return False

if __name__ == "__main__":
    IMG_DIR = "D:\벼이삭 모델링\dataset_pascal/JPEGImages"
    XML_DIR = "D:\벼이삭 모델링\dataset_pascal/Annotations"
    AUG_XML_DIR = "D:\벼이삭 모델링\dataset_pascal/AUG_XML"  # 향상된 xml 폴더 경로 저장
    mkdir(AUG_XML_DIR)
    AUG_IMG_DIR = "D:\벼이삭 모델링\dataset_pascal/AUG_IMG"  # 향상된 이미지 폴도 경로 저장
    mkdir(AUG_IMG_DIR)
    AUGLOOP = 10 # 이미지당 보정횟수
    boxes_img_aug_list = []
    new_bndbox = []
    new_bndbox_list = []


    seq = iaa.Sequential([
        iaa.Flipud(0.5),  # 모든 이미지의 50%를 수직으로 뒤집기
        iaa.Fliplr(0.5),  # Mirror
        iaa.Multiply((1.2, 1.5)),  # change brightness, doesn't affect BBs
        iaa.GaussianBlur(sigma=(0, 3.0)), # iaa.GaussianBlur(0.5),
        iaa.Affine(
            translate_px={"x": 15, "y": 15},
            scale=(0.8, 0.95),
            rotate=(-30, 30)
        )  # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
    ])

    for root, sub_folders, files in os.walk(XML_DIR):

        for name in files:

            bndbox = read_xml_annotation(XML_DIR, name)

            for epoch in range(AUGLOOP):
                seq_det = seq.to_deterministic()  

                img = Image.open(os.path.join(IMG_DIR, name[:-4] + '.jpg'))
                img = np.array(img)

                for i in range(len(bndbox)):
                    bbs = ia.BoundingBoxesOnImage([
                        ia.BoundingBox(x1=bndbox[i][0], y1=bndbox[i][1], x2=bndbox[i][2], y2=bndbox[i][3]),
                    ], shape=img.shape)

                    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
                    boxes_img_aug_list.append(bbs_aug)

                    # new_bndbox_list:[[x1,y1,x2,y2],...[],[]]
                    new_bndbox_list.append([int(bbs_aug.bounding_boxes[0].x1),
                                            int(bbs_aug.bounding_boxes[0].y1),
                                            int(bbs_aug.bounding_boxes[0].x2),
                                            int(bbs_aug.bounding_boxes[0].y2)])

                image_aug = seq_det.augment_images([img])[0]
                path = os.path.join(AUG_IMG_DIR, str(name[:-4]) + "_aug_" + str(epoch) + '.jpg')
                Image.fromarray(image_aug).save(path)


                change_xml_list_annotation(XML_DIR, name[:-4], new_bndbox_list,AUG_XML_DIR,epoch)
                print(str(name[:-4]) + "_aug_" + str(epoch) + '.jpg')
                new_bndbox_list = []

D:\벼이삭 모델링\dataset_pascal/AUG_XML성공
D:\벼이삭 모델링\dataset_pascal/AUG_IMG성공
20210913_222443_aug_0.jpg
20210913_222443_aug_1.jpg
20210913_222443_aug_2.jpg
20210913_222443_aug_3.jpg
20210913_222443_aug_4.jpg
20210913_222443_aug_5.jpg
20210913_222443_aug_6.jpg
20210913_222443_aug_7.jpg
20210913_222443_aug_8.jpg
20210913_222443_aug_9.jpg
20210913_222520_aug_0.jpg
20210913_222520_aug_1.jpg
20210913_222520_aug_2.jpg
20210913_222520_aug_3.jpg
20210913_222520_aug_4.jpg
20210913_222520_aug_5.jpg
20210913_222520_aug_6.jpg
20210913_222520_aug_7.jpg
20210913_222520_aug_8.jpg
20210913_222520_aug_9.jpg
20210913_222814_aug_0.jpg
20210913_222814_aug_1.jpg
20210913_222814_aug_2.jpg
20210913_222814_aug_3.jpg
20210913_222814_aug_4.jpg
20210913_222814_aug_5.jpg
20210913_222814_aug_6.jpg
20210913_222814_aug_7.jpg
20210913_222814_aug_8.jpg
20210913_222814_aug_9.jpg
20210913_222853_aug_0.jpg
20210913_222853_aug_1.jpg
20210913_222853_aug_2.jpg
20210913_222853_aug_3.jpg
20210913_222853_aug_4.jpg
20210913_222853_au

# 코드 설명 

### 원본 이미지 경계 상자 좌표 읽기
#### xml 파일을 읽고 ElementTree를 사용하여 xml파일을 파싱하여 각 개체의 좌표값을 찾음

In [33]:
def read_xml_annotation(root, image_id):
    in_file = open(os.path.join(root, image_id), encoding='UTF8')
    tree = ET.parse(in_file)
    root = tree.getroot()
    bndboxlist = []

    for object in root.findall('object'):  # 루느 노드 아래의 모든 객체 노드 bound box 찾기 
        bndbox = object.find('bndbox')  #자식 노드에서 노드 순위의 값

        xmin = int(bndbox.find('xmin').text)
        xmax = int(bndbox.find('xmax').text)
        ymin = int(bndbox.find('ymin').text)
        ymax = int(bndbox.find('ymax').text)
        # print(xmin,ymin,xmax,ymax)
        bndboxlist.append([xmin,ymin,xmax,ymax])
        # print(bndboxlist)

    bndbox = root.find('object').find('bndbox')
    return bndboxlist

### 변환된 경계 상자 좌표 파일 생성
#### 변환된 대상의 경계 상자 좌표를 전달하고 원래 좌표를 새 좌표로 바꾸고 새 xml파일을 생성

In [34]:
def change_xml_list_annotation(root, image_id, new_target,saveroot,id):

    in_file = open(os.path.join(root, str(image_id) + '.xml'), encoding='UTF8')  #여기서 두개의 루트는 각각 Tree=ET.parse(in_file)
    tree = ET.parse(in_file)
    xmlroot = tree.getroot()
    index = 0

    for object in xmlroot.findall('object'):  # 루트 노드 아래에 있는 모든 노드 찾기
        bndbox = object.find('bndbox')  

        new_xmin = new_target[index][0]
        new_ymin = new_target[index][1]
        new_xmax = new_target[index][2]
        new_ymax = new_target[index][3]

        xmin = bndbox.find('xmin')
        xmin.text = str(new_xmin)
        ymin = bndbox.find('ymin')
        ymin.text = str(new_ymin)
        xmax = bndbox.find('xmax')
        xmax.text = str(new_xmax)
        ymax = bndbox.find('ymax')
        ymax.text = str(new_ymax)

        index = index + 1

    tree.write(os.path.join(saveroot, str(image_id) + "_aug_" + str(id) + '.xml'))

### 변환 시퀀스 생성 

In [35]:
seq = iaa.Sequential([
        iaa.Flipud(0.5),  # 모든 이미지의 20%를 수직으로 뒤집기
        iaa.Fliplr(0.5),  # MIrror
        iaa.GaussianBlur(sigma=(0, 3.0)),
        # iaa.GaussianBlur(0.5),
        iaa.Affine(
            translate_px={"x": 15, "y": 15},
            scale=(0.8, 0.95),
            rotate=(-30, 30))
    ])

### 경계 상자 변경후 좌표 계산
#### 먼저 이미지에 해당하는 xml파일을 읽고 모든 대상의 경계상자를 가져온 다음 각 상자의 변경된 좌표를 차례로 계산

In [44]:
bndbox = read_xml_annotation(XML_DIR, name)
for epoch in range(AUGLOOP):
    seq_det = seq.to_deterministic()
    img = Image.open(os.path.join(IMG_DIR, name[:-4] + '.jpg'))
    img = np.array(img)
    for i in range(len(bndbox)):
        bbs = ia.BoundingBoxesOnImage([
        ia.BoundingBox(x1=bndbox[i][0], y1=bndbox[i][1], x2=bndbox[i][2], y2=bndbox[i][3]),
        ], shape=img.shape)

        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
        boxes_img_aug_list.append(bbs_aug)

In [37]:
def mkdir(path):
    path = path.strip()
    path = path.rstrip("\\")
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
        print(path + '성공')
        return True
    else:
        print(path + '이미 존재')
        return False

### 데이터 준비
#### 입력 데이터는 Annotations (xml파일), JPEGImages(jpg이미지)

In [38]:
IMG_DIR = "D:\벼이삭 모델링\dataset_pascal/JPEGImages" #이미지 폴더 경로
XML_DIR = "D:\벼이삭 모델링\dataset_pascal/Annotations" # xml폴더 경로 입력


AUG_XML_DIR = "D:\벼이삭 모델링\dataset_pascal/AUG_XML" # 향상된 xml 폴더 경로 저장
mkdir(AUG_XML_DIR)

AUG_IMG_DIR = "D:\벼이삭 모델링\dataset_pascal/AUG_IMG" # 향상된 이미지 폴도 경로 저장
mkdir(AUG_IMG_DIR)

D:\벼이삭 모델링\dataset_pascal/AUG_XML이미 존재
D:\벼이삭 모델링\dataset_pascal/AUG_IMG이미 존재


False

In [39]:
 AUGLOOP = 10 # 이미지당 보정 횟수

In [40]:
seq = iaa.Sequential([
        iaa.Flipud(0.5),  # 모든 이미지의 50%를 수직으로 뒤집기
        iaa.Fliplr(0.5),  # Mirror
        iaa.Multiply((1.2, 1.5)),  # 밝기 변경, 영향을 주지 않음 BB
        iaa.GaussianBlur(sigma=(0, 0.5)),
         # iaa.GaussianBlur(0.5),
        iaa.Affine(
            translate_px={"x": 15, "y": 15},
            scale=(0.8, 0.95),
            rotate=(-30, 30)
        )  
    ])